# Setup

In [ ]:
# import libraries
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from matplotlib import image
from tensorflow.keras import layers
from keras.models import load_model
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# unzip the training zip folder
!unzip training.zip

# Preprocessing dataset

In [ ]:
# resize every image on the dataset to 299x299 
for folder_name in os.listdir(os.path.relpath("training")):
  if not folder_name.endswith("Store"):
      for fname in os.listdir(os.path.join("training", folder_name)):
        if not fname.endswith(".jpg"):
          os.remove(os.path.join("training", folder_name, fname))
        else:
          img = Image.open(os.path.join("training", folder_name, fname))
          img = img.resize((299, 299), Image.ANTIALIAS)
          img.save(os.path.join("training", folder_name, fname))
  else:
     os.remove(os.path.join("training", folder_name))

# Train and test dataset

In [ ]:
# Split the dataset into a training and test dataset
image_size = (299, 299)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "training",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "training",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

# Model

In [ ]:
# Build the model
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal")]
)

base_model.trainable = False
inputs = keras.Input(shape=(299, 299, 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.6)(x)
outputs = keras.layers.Dense(15)(x)
model = keras.Model(inputs, outputs)

# Training

In [ ]:
# Train the model using the transfer learning and fine tuning techniques
epochs = 50

callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

# Unfreeze the base model
base_model.trainable = True
callbacks2 = [keras.callbacks.ModelCheckpoint("save2_at_{epoch}.h5")]
model.compile(#optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              optimizer=keras.optimizers.RMSprop(1e-5),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.fit(
    train_ds, epochs=epochs, callbacks=callbacks2, validation_data=val_ds,
)

# Preprocessing Test dataset

In [ ]:
# unzip test dataset
!unzip testing.zip

In [ ]:
# resize each image to 299x299
for fname in os.listdir("testing"):
  if not fname.endswith(".jpg"):
    os.remove(os.path.join("testing", fname))
  else:
    img = Image.open(os.path.join("testing", fname))
    img = img.resize((299, 299), Image.ANTIALIAS)
    img.save(os.path.join("testing", fname))

# Inference

In [ ]:
# Dictionary for the classes
map = {
    0 : "Coast",
    1 : "Forest",
    2 : "Highway",
    3 : "Insidecity",
    4 : "Mountain",
    5 : "Office",
    6 : "OpenCountry",
    7 : "Street",
    8 : "Suburb",
    9 : "TallBuilding",
    10 : "bedroom",
    11 : "industrial",
    12 : "kitchen",
    13 : "livingroom",
    14 : "store"
}

In [ ]:
# Infer the class of each image on the test dataset.
# Save the image names and the corresponding predictions on a list.
# Sort that list into ascending order based on the name of the image.  
image_size = (299,299)
l = []
for fname in os.listdir("testing"):
  print(fname)
  img = keras.preprocessing.image.load_img(
      os.path.join("testing", fname), target_size=image_size
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis

  predictions = model.predict(img_array)
  score = predictions[0]
  label = map[score.argmax()]
  l.append((int(fname.replace(".jpg","")), label))

l.sort()

In [ ]:
# Save the content of the list into a text file called run3.txt
with open('run3.txt', 'w') as f:
  for fname,label in l:
    f.write(f"{fname}.jpg {label}\n")
